In [2]:
!pip install -q geopandas
!apt install -q proj-bin libproj-dev libgeos-dev -y
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip
!pip install -q rasterio

# Pandas is a package containing additional functions to use data frames in Python
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.basemap import Basemap
import warnings
import rasterio
import numpy as np
import seaborn as sns
warnings.simplefilter('ignore')
# These two lines allow the notebook to access the Google Drive.
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# This is the path to the project folder within the Google Drive.
file_path = "/content/drive/My Drive/"
import rpy2.interactive

%load_ext rpy2.ipython

from rpy2.robjects import pandas2ri  # activate pandas R  interface
pandas2ri.activate()

Reading package lists...
Building dependency tree...
Reading state information...
libgeos-dev is already the newest version (3.6.2-1build2).
libproj-dev is already the newest version (4.9.3-2).
proj-bin is already the newest version (4.9.3-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Mounted at /content/drive


Here we load some packages in another programming language - R. We won't use this language very much (because I prefer Python), but we will briefly switch languages to run the model because the tool to run it requires this language.

Running this cell might take a a couple of minutes.

In [3]:
 %%R
install.packages('rgeos', quiet=TRUE)
install.packages('raster', quiet=TRUE)
install.packages('rgdal', quiet=TRUE)
install.packages('maptools', quiet=TRUE)
install.packages('dismo', quiet=TRUE)

require('rgdal', quiet=TRUE)
require('rgeos', quiet=TRUE)
require('maptools', quiet=TRUE)
require('dismo', quiet=TRUE)
require('raster', quiet=TRUE)


R[write to console]: rgeos version: 0.5-3, (SVN revision 634)
 GEOS runtime version: 3.6.2-CAPI-1.10.2 
 Linking to sp version: 1.4-2 
 Polygon checking: TRUE 


R[write to console]: Checking rgeos availability: TRUE



In [4]:
%%R
install.packages('randomForest', quiet=TRUE)
require('randomForest', quiet=TRUE)


R[write to console]: randomForest 4.6-14

R[write to console]: Type rfNews() to see new features/changes/bug fixes.



In [5]:
def convert_xy_to_longlat(grid_x, grid_y):
  lon = ((grid_x / 6) - 180)
  lat = -((grid_y / 6) - 90)
  return (lon, lat)

def convert_longlat_to_xy(lon, lat):
  grid_x = int((lon + 180) * 6)
  grid_y = int((-lat + 90) * 6)
  return (grid_x, grid_y)

In [6]:
import os

---
## Notebook 15
# Applying Species Distribution Models - Automated
In this notebook we repeat the code from notebook 14 but it is run on every species and on every combination of models, ssps and time periods.

In [11]:
species_list = [line.strip() for line in open(file_path + "species_names.tsv")]
models = ['BCC-CSM2-MR',
         'CanESM5']
scenarios = ['ssp126', 'ssp585']

time_periods = ['2021-2040', '2041-2060', '2061-2080', '2081-2100']

this_boundary = [((0, 2160)), ((930, 0))]

# make a dictionary of bioclim variable names for convinence
bioclim = pd.read_csv(file_path + "bioclim.tsv", sep="\t")
bioclim_name = dict(zip(bioclim['variable_number'], bioclim['name']))

# make a list of bioclim variables 
bc = list(bioclim_name.values())

for species_name in species_list:
      if not os.path.exists(file_path + "SDM_results/" + species_name):
        os.mkdir(file_path + "SDM_results/" + species_name)
        print (species_name)
        if os.path.exists(file_path + "species_plus_climate_observed_geo/" + species_name + ".tsv"):
          # read the presence data
          presence_table = pd.read_csv(file_path + "species_plus_climate_observed_geo/" + species_name + ".tsv", sep="\t")

          if len(presence_table) > 50:
            # add a column to show these points are where the species is present
            presence_table['p'] = 1

            # read the absence data
            try:
              absence_table = pd.read_csv(file_path + "species_plus_climate_observed_absence/" + species_name + "_geo.tsv", sep="\t")
            except:
              continue
            absence_table['p'] = 0
            # Calculate the sample size for the presence and absence data - 5% of the total number of records
            sample_size_presence = int(len(presence_table) * 0.05)
            sample_size_absence = int(len(absence_table) * 0.05)

            # take a random 5% of observation IDs for the presence data
            sample_presence = np.random.choice(presence_table['obs_ID'], sample_size_presence)

            # take a random 5% of observation IDs for the absence data
            sample_absence = np.random.choice(absence_table['obs_ID'], sample_size_absence)

            # extract the 5% of samples for testing
            training_presence = presence_table[~presence_table['obs_ID'].isin(sample_presence)]
            training_absence = absence_table[~absence_table['obs_ID'].isin(sample_absence)]


            # extract the 95% of samples for training
            testing_presence = presence_table[presence_table['obs_ID'].isin(sample_presence)]
            testing_absence = absence_table[absence_table['obs_ID'].isin(sample_absence)]

            # combine the testing and training data for presence and absence
            testing_combined = testing_presence.append(testing_absence)
            training_combined = training_presence.append(training_absence)

            # remove the extra columns (with non-climate data e.g. latitude and longitude)
            # from the testing and training tables
            testing_combined = testing_combined[list(bioclim_name.values()) + ['p']].astype(float)
            training_combined = training_combined[list(bioclim_name.values()) + ['p']].astype(float)
            # set the paths to the raster files for current and predicted climate
            raster_path_current = file_path + "climate_data/" + "near_present.tif"

            raster_current = rasterio.open(raster_path_current)
            # convert the data into a matrix, round to 6dp, replace inf with nan
            grid_current = raster_current.read()
            grid_current = np.round(grid_current, 6)
            grid_current[grid_current == float('-inf'), ] = float('nan')
            # exclude very low latitudes
            gc = grid_current[:, this_boundary[1][1]:this_boundary[1][0], this_boundary[0][0]:this_boundary[0][1]]

            # pass the dataframes in to R
            %R -i testing_combined
            %R -i training_combined

            # pass the list of Bioclim variables into R
            %R -i bc
          # make an empty R raster object
            %R raster_current = raster()

            # for each layer of the raster grid
            for i in np.arange(0, 19, 1):
                # extract a single layer from the Python raster
                grid_current = gc[i]
                # pass the layer to R
                %R -i grid_current
                # convert to a matrix
                %R grid_matrix_current = as.matrix(grid_current)

                # find the minimum and maximum dimensions
                %R xmax_current = dim(grid_current)[1]
                %R ymax_current = dim(grid_current)[2]

                # convert to an R raster layer
                %R this_grid_current_raster = raster(grid_matrix_current, xmn=0, xmx=xmax_current, ymn=0, ymx=ymax_current, template=NULL)
                # add the layer to the big R raster
                %R raster_current = addLayer(raster_current, this_grid_current_raster)


            # name the layers of the R raster using the Bioclim variable names
            %R names(raster_current) = bc

            # make copies of all the testing tables without the presence / absence column
            testing_presence_bc = testing_presence[bc]
            testing_absence_bc = testing_absence[bc]
            training_presence_bc = training_presence[bc]
            testing_combined_bc = testing_combined[bc]

            # pass all the tables into R
            %R -i testing_presence_bc
            %R -i testing_absence_bc
            %R -i training_presence_bc
            %R -i testing_combined_bc
            %R bc = unlist(bc)
            # Create a GLM model based on the training data
            %R glm_model = glm(p ~ . , data=training_combined, family=binomial(logit))

            # Create a Bioclim model based on the training data
            %R bioclim_model = bioclim(training_presence_bc)

            # for Bioclim we also need to find a threshold to assign species as present or absent
            %R e_bioclim = evaluate(testing_presence_bc, testing_absence_bc, bioclim_model)
            %R t_bioclim <- threshold(e_bioclim, 'spec_sens')

            # Create a random forest model based on the training data
            %R rf_model = randomForest(factor(p) ~ ., data=training_combined)


            # predict presence or absence for the test data using the GLM model
            %R glm_test = predict(glm_model, testing_combined_bc, type='response') > 0.5

            # predict presence or absence for the test data using the Bioclim Model
            %R bioclim_test = predict(bioclim_model, testing_combined_bc) > t_bioclim

            # predict presence or absence for the test data using the random forest model
            %R rf_test = predict(rf_model, testing_combined_bc)

            %R -o glm_test
            %R -o bioclim_test
            %R -o rf_test
            %R -o t_bioclim
            testing_combined['test_result_glm'] = glm_test
            testing_combined['test_result_bioclim'] = bioclim_test
            testing_combined['test_result_rf'] = rf_test.astype(int) 

            # Count how many times the model correctly predicted presence or absence for the test data
            total_correct_glm = sum(testing_combined['test_result_glm'] == testing_combined['p'])
            total_correct_bioclim = sum(testing_combined['test_result_bioclim'] == testing_combined['p'])
            total_correct_rf = sum(testing_combined['test_result_rf'] == testing_combined['p'])

            # convert these to percentages
            percent_correct_glm = round((total_correct_glm / len(testing_combined)) * 100, 2)
            percent_correct_bioclim = round((total_correct_bioclim / len(testing_combined)) * 100, 2)
            percent_correct_rf = round((total_correct_rf / len(testing_combined)) * 100, 2)


            testing_combined.to_csv(file_path + "SDM_results/" + species_name + "testing.tsv", sep="\t", index=None)
            # Predict current habitable areas under the glm model
            %R res_glm = predict(raster_current, glm_model, type='response')
            %R res_glm = as.matrix(res_glm[[1]])

            # Predict current habitable areas under the bioclim model
            %R res_bioclim = predict(raster_current, bioclim_model)
            %R res_bioclim = as.matrix(res_bioclim[[1]])

            # Predict current habitable areas under the random forest model
            %R res_rf = predict(raster_current, rf_model)
            %R res_rf = as.matrix(res_rf[[1]])

            %R -o res_glm
            %R -o res_bioclim
            %R -o res_rf

            D = dict()

            D['present_glm'] = res_glm
            D['present_bioclim'] = res_bioclim
            D['present_rf'] = res_rf

            for key in D:
              # make sure all the values are floats
              D[key] = D[key].astype('float')

              # fill in NA where appropriate
              D[key][D[key] < 0] = float('nan')
              if "glm" in key:
                # glm predictions should be normalised to 0 or 1
                D[key][D[key] > 0.5] = 1
                D[key][(D[key] < 0.5) & (D[key] > 0)] = 0
              elif "bioclim" in key:
                # bioclim predictions should be assigned using the bioclim threshold
                D[key][D[key] > t_bioclim] = 1
                D[key][(D[key] < t_bioclim) & (D[key] > 0)] = 0


            np.save(file_path + "SDM_results/" + species_name + "/" + "present_glm.npy", D['present_glm'])
            np.save(file_path + "SDM_results/" + species_name + "/" + "present_bioclim.npy", D['present_bioclim'])
            np.save(file_path + "SDM_results/" + species_name + "/" + "present_rf.npy", D['present_rf'])
            for model_future in models:
              for ssp_future in scenarios:
                for timepoint_future in time_periods:
                  D2 = dict()
                  # Print a message so we know it's still running
                  print ("Importing data for %s %s %s" % (model_future, ssp_future, timepoint_future))

                  # generate the path to the raster data
                  raster_path_future = file_path + "climate_data/" + model_future + "/" + ssp_future + "/" + timepoint_future + ".tiff"

                  # read the raster into Python and tidy it up
                  raster_future = rasterio.open(raster_path_future)
                  grid_future = raster_future.read()
                  grid_future = np.round(grid_future, 6)
                  grid_future[grid_future == float('-inf'), ] = float('nan')
                  # exclude very low latitudes
                  gf = grid_future[:, this_boundary[1][1]:this_boundary[1][0], this_boundary[0][0]:this_boundary[0][1]]

                  # generate an empty R raster
                  %R raster_future = raster()
                  # for each layer of the Python raster
                  for i in range(0, 19):
                      # extract the current layer from the Python raster
                      grid_future = gf[i]

                      # pass the layer into R
                      %R -i grid_future
                      # convert to a matrix
                      %R grid_matrix_future = as.matrix(grid_future)

                      # find the minimum and maximum dimensions
                      %R xmax_future = dim(grid_future)[1]
                      %R ymax_future = dim(grid_future)[2]

                      # convert to an R raster layer
                      %R this_grid_future_raster = raster(grid_matrix_future, xmn=0, xmx=xmax_future, ymn=0, ymx=ymax_future, template=NULL)
                      # add the layer to the big R raster
                      %R raster_future = addLayer(raster_future, this_grid_future_raster)
                  # rename the R raster layers using the R variable names
                  %R names(raster_future) = bc
                  print ("Exporting data for %s %s %s" % (model_future, ssp_future, timepoint_future)) 


                  # predict the habitable area using the GLM model
                  %R res_glmx = predict(raster_future, glm_model, type='response')
                  %R res_glmx = as.matrix(res_glmx[[1]])

                  # predict the habitable area using the bioclim model
                  %R res_bioclimx = predict(raster_future, bioclim_model)
                  %R res_bioclimx = as.matrix(res_bioclimx[[1]])

                  # predict the habitable area using the random forest model
                  %R res_rfx = predict(raster_future, rf_model)
                  %R res_rfx = as.matrix(res_rfx[[1]])

                  # output everything back to Python
                  %R -o res_glmx
                  %R -o res_bioclimx
                  %R -o res_rfx
                  subnam = model_future + "_" + ssp_future + "_" + timepoint_future + "_"
                  D2[subnam + "_glm"] = res_glmx
                  D2[subnam + "_bioclim"] = res_bioclimx
                  D2[subnam + "_rf"] = res_rfx 
                  for key in D2:
                    # make sure all the values are floats
                    D2[key] = D2[key].astype('float')

                    # fill in NA where appropriate
                    D2[key][D2[key] < 0] = float('nan')
                    if "glm" in key:
                      # glm predictions should be normalised to 0 or 1
                      D2[key][D2[key] > 0.5] = 1
                      D2[key][(D2[key] < 0.5) & (D2[key] > 0)] = 0
                    elif "bioclim" in key:
                      # bioclim predictions should be assigned using the bioclim threshold
                      D2[key][D2[key] > t_bioclim] = 1
                      D2[key][(D2[key] < t_bioclim) & (D2[key] > 0)] = 0

                  np.save(file_path + "SDM_results/" + species_name + "/" + subnam +  "_glm.npy", D2[subnam + '_glm'])
                  np.save(file_path + "SDM_results/" + species_name + "/" + subnam + "_bioclim.npy", D2[subnam + '_bioclim'])
                  np.save(file_path + "SDM_results/" + species_name + "/" + subnam + "_rf.npy", D2[subnam + '_rf'])


Ceratina_australensis
Rhizoglyphus_robini
Solenopsis_invicta
Importing data for BCC-CSM2-MR ssp126 2021-2040
Exporting data for BCC-CSM2-MR ssp126 2021-2040
Importing data for BCC-CSM2-MR ssp126 2041-2060
Exporting data for BCC-CSM2-MR ssp126 2041-2060
Importing data for BCC-CSM2-MR ssp126 2061-2080
Exporting data for BCC-CSM2-MR ssp126 2061-2080
Importing data for BCC-CSM2-MR ssp126 2081-2100
Exporting data for BCC-CSM2-MR ssp126 2081-2100
Importing data for BCC-CSM2-MR ssp585 2021-2040
Exporting data for BCC-CSM2-MR ssp585 2021-2040
Importing data for BCC-CSM2-MR ssp585 2041-2060
Exporting data for BCC-CSM2-MR ssp585 2041-2060
Importing data for BCC-CSM2-MR ssp585 2061-2080
Exporting data for BCC-CSM2-MR ssp585 2061-2080
Importing data for BCC-CSM2-MR ssp585 2081-2100
Exporting data for BCC-CSM2-MR ssp585 2081-2100
Importing data for CanESM5 ssp126 2021-2040
Exporting data for CanESM5 ssp126 2021-2040
Importing data for CanESM5 ssp126 2041-2060
Exporting data for CanESM5 ssp126 2041-